In [1]:
import math
import numpy
import nagisa
from collections import Counter
from bs4 import BeautifulSoup
import MeCab
import gensim
from scipy import spatial
import re
import sys
import codecs
import numpy as np
from gensim.models.word2vec import Word2Vec
import requests
from scipy.stats import norm, entropy
from collections import namedtuple
import glob
import CaboCha
import jctconv, os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from pykakasi import kakasi
import unicodedata
import random

In [6]:
#ファイルの取得

pass1=glob.glob("../text/*")#ここのパスは個々人で書き換え
pass2=glob.glob(pass1[0]+"/*")

texts=[]
texts_title=[]
texts_original=[]
for i in pass1:
    pass2=glob.glob(i+"/*")
    for i1 in pass2:
        with open(i1) as f:
            areas = f.readlines() 
            #いらない部分を削除
        areas.pop(0)
        areas.pop(0)
        #タイトルの空白は削除
        ls="".join(areas[0].split())
        #テキストタイトルの取得
        texts_title.append(ls)
        #タイトルの削除
        areas.pop(0)
        areas = ''.join(areas)
        texts.append(areas)
        texts_original.append(areas)
        

In [7]:
#テキストの空白削除

for i in range(len(texts)):
    texts[i]="".join(texts[i].split())

In [8]:
#使用する文章の作成(get_sentencesで区切る)

POS_DIC = {
    'BOS/EOS': 'EOS', # end of sentense
    '形容詞' : 'ADJ',
    '連体詞' : 'JADJ', # Japanese-specific POS like a adjective
    '副詞'   : 'ADV',
    '名詞'   : 'NOUN',
    '動詞'   : 'VERB',
    '助動詞' : 'AUX',
    '助詞'   : 'PART',
    '感動詞' : 'INTJ',
    '接続詞' : 'CONJ',
    '記号'   : 'SYM', # symbol
    '*'      : 'X',
    'その他' : 'X',
    'フィラー': 'X',
    '接頭詞' : 'X',
}
POS_2ND_DIC = {
    '代名詞':'PRON',
}

def get_sentences(text):
    """ 
    input: text have many sentences
    output: ary of sentences ['sent1', 'sent2', ...]
    """
    EOS_DIC = ['。', '．', '！','？','!?', '!', '?' ]
    sentences = list()#からのリスト
    sent = ''#空のリスト
    # split in first when text include '\n'
    temp = text.split('\\n')#tempにテキストを空白文字で区切ったもの
    for each_text in temp:#tempでfor
        if each_text == '':#からならば
            continue
        for token in tokenize(each_text):
            # print(token.pos_jp, token.pos, token.surface, sent)
            # TODO: this is simple way. ex)「今日は雨ね。」と母がいった
            sent += token.surface#sentに表層形を足す
            if token.surface in EOS_DIC and sent != '':#EOS_DICの中にtoken.surfaceがあり、sentがからでなければ
                sentences.append(sent)#sentencesにsentをappend
                sent = ''#sentを空に
        if sent != '':#sentがからでなければ
            sentences.append(sent)#sentencesにsentをappend
    return sentences

def tokenize(sent):
    tagger = MeCab.Tagger ('-Ochasen /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
    tagger.parse('') 
    node = tagger.parseToNode( sent )
    #print(sent)
    tokens = list()
    idx = 0
    while node:
        #print(node.surface)
        feature = node.feature.split(',')
        token = namedtuple('Token', 'idx, surface, pos, pos_detail1, pos_detail2, pos_detail3,infl_type, infl_form, base_form, reading, phonetic')
        token.idx         = idx
        token.surface     = node.surface  # 表層形
        token.pos_jp      = feature[0]    # 品詞
        token.pos_detail1 = feature[1]    # 品詞細分類1
        token.pos_detail2 = feature[2]    # 品詞細分類2
        token.pos_detail3 = feature[3]    # 品詞細分類3
        token.infl_type   = feature[4]    # 活用型
        token.infl_form   = feature[5]    # 活用形
        token.base_form   = feature[6] if feature[6]!='*' else node.surface # 原型 ex)MacMini's base_form=='*'
        token.pos         = POS_DIC.get( feature[0], 'X' )     # 品詞
        token.reading     = feature[7] if len(feature) > 7 else ''  # 読み
        token.phonetic    = feature[8] if len(feature) > 8 else ''  # 発音
        # for BOS/EOS
        if token.pos != 'EOS':#posがEOSでない場合
            tokens.append(token)#tokensにtokenをappend
            idx += 1#idxを+1(分かち書きの文節？の数)
        node = node.next#次の
    return tokens

def normalize(src_text):
    # Zenkaku to Hankaku ( handling japaneze character )
    normalized = jctconv.h2z(src_text, digit=False, ascii=False) #半角を全角に
    normalized = jctconv.z2h(normalized, kana=False)#全角を半角に
    return normalized.lower()#小文字に変換して返す

In [9]:
#全ての文章をget_sentencesで分ける
#text_s=[]
text_use=[]
for tex in tqdm(texts):
    #tex = ''.join(tex.split()) 
    text_use.append(get_sentences(tex))
    #text_s.append(text)
#print(text)

In [10]:
#必要なし、5単語より短い文を削除しようとしてやめた
text_backup1=text_use
text_backup2= texts_original
text_backup3= texts_title
for y in range(len(text_use)):
    if len(text_use[y])<=5:
        text_use.pop(y)
        texts_original.pop(y)
        texts_title.pop(y)


IndexError: list index out of range

In [11]:
print(len(text_use))
print(len(texts_original))

7213
7213


In [12]:
#ma_parceでtopicの算出に用いるための名詞の抽出を行う結果はwordsとしてリスト化


mecab = MeCab.Tagger("-Ochasen -d /usr/local/mecab/lib/mecab/dic/mecab-ipadic-neologd/")
mecab.parse('')
# MeCabを使って形態素解析をします。
def ma_parse(sentence, filter="名詞",filter2="名詞,非自立",filter3="名詞,固有",filter4="名詞,代名詞",filter5="名詞,接尾",filter6="名詞,数"):
    node = mecab.parseToNode(sentence)
    while node:
        if node.feature.startswith(filter) and not (node.feature.startswith(filter2))  and not (node.feature.startswith(filter4)) and not (node.feature.startswith(filter5)) and not (node.feature.startswith(filter6)) and node.surface!="自分":
            yield node.surface
        node = node.next

words=[]
words_use_corpus=[]
for i in range(len(text_use)):
    word=[]
    for s in range(len(text_use[i])):
        words_use_corpus.append([word for word in ma_parse(text_use[i][s])]) 
        word.append([word for word in ma_parse(text_use[i][s])])
    words.append(word)
#print(words_use_corpus)
#print(word)

In [13]:
#ldaでそれぞれの文章のtopicの元になるldaを求める
topic_number=8
from gensim import models, corpora
dictionary = corpora.dictionary.Dictionary(words_use_corpus)
corpus = [dictionary.doc2bow(count) for count in words_use_corpus]
#print(corpus)
lda = models.ldamodel.LdaModel(corpus=corpus, num_topics=topic_number, id2word=dictionary)

# Topics
#for topic in lda.show_topics(-1):
    #print('topic')
    #print(topic)

# Topic of each document
#for topics_per_document in lda[corpus]:
    #print('topic of ecah document')
    #print(topics_per_document)

In [23]:
#話し言葉変換のための辞書作成(endが終助詞、hukuが副詞,keiyouが形容詞,setuが接続詞,sonotaがその他（いらないかもしれない）,zyoshiが助詞,meishiが名詞)
pd.set_option('display.max_columns', 10000)#ディスプレイにおいて表示されるpandasの数
pd.set_option('display.max_rows', 10000)#上記と同じ
#df_none = pd.read_csv('pn.csv', sep="\t",names=('A', 'B','C'))#pandasで読み込み
pn_end=pd.read_table('./for_conversion/sent_talk_end.txt',names=('A','B'),sep=",")#同上
pn_huku=pd.read_table('./for_conversion/sent_talk_huku.txt',names=('A','B'),sep=",")#同上
pn_keiyou=pd.read_table('./for_conversion/sent_talk_keiyou.txt',names=('A','B'),sep=",")#同上
pn_setu=pd.read_table('./for_conversion/sent_talk_setu.txt',names=('A','B'),sep=",")#同上
pn_sonota=pd.read_table('./for_conversion/sent_talk_sonota1.txt',names=('A','B'),sep=",")#同上
pn_zyoshi=pd.read_table('./for_conversion/sent_talk_zyosi.txt',names=('A','B'),sep=",")#同上
pn_meishi=pd.read_table('./for_conversion/sent_talk_meishi.txt',names=('A','B'),sep=",")#同上
        #s=df_none.spit("¥t",expand=True)
        #pn=pn.T
#df_none_dict= dict([(i,a) for i, a in zip(df_none.A,df_none.B)])
#話し言葉変換の辞書作成
pn_answer_end=dict([(a,i) for i, a in zip(pn_end.A,pn_end.B)])
pn_answer_huku=dict([(a,i) for i, a in zip(pn_huku.A,pn_huku.B)])
pn_answer_keiyou=dict([(a,i) for i, a in zip(pn_keiyou.A,pn_keiyou.B)])
pn_answer_setu=dict([(a,i) for i, a in zip(pn_setu.A,pn_setu.B)])
pn_answer_sonota=dict([(a,i) for i, a in zip(pn_sonota.A,pn_sonota.B)])
pn_answer_zyoshi=dict([(a,i) for i, a in zip(pn_zyoshi.A,pn_zyoshi.B)])
pn_answer_meishi=dict([(a,i) for i, a in zip(pn_meishi.A,pn_meishi.B)])
#pn_answer_end.update(pn_answer_huku)#pn_answer連結
#pn_answer_end.update(pn_answer_keiyou)
#pn_answer_end.update(pn_answer_setu)
#pn_answer_end.update(pn_answer_sonota)
#pn_answer_end["など"]="とか"
#print(pn_answer_end)

In [24]:
#絵文字辞書作成
pd.set_option('display.max_columns', 10000)#ディスプレイにおいて表示されるpandasの数
pd.set_option('display.max_rows', 10000)#上記と同じ
#p=list(range(40))
#print(p)
with codecs.open("./for_conversion/kazyo_emoji.txt", "r", "Shift-JIS", "ignore") as file:
    df=pd.read_table(file,names=('A','B','C'))
    #df = pd.read_table(file, delimiter=",")
df_none1 = pd.read_csv('./for_conversion/kanzyo_emoji1.csv')#pandasで読み込み
df_none2=pd.read_csv('./for_conversion/kanzyo_emoji2.csv')#pandasで読み込み
df_none3=pd.read_csv('./for_conversion/kanzyo_emoji3.csv')
kanzyo_emoji_dict1= dict([(i,a) for i, a in zip(df_none1.Word,df_none1.Emotion)])
kanzyo_emoji_dict2=dict([(i,a) for i, a in zip(df_none2.Word,df_none2.Emotion)])
kanzyo_emoji_dict3=dict([(i,a) for i, a in zip(df_none3.Word,df_none3.Emotion)])
emoji_dict=dict([(i,a) for i, a in zip(df.B,df.C)])

In [16]:
#絵文字辞書を絵文字と感情のラベルと、ラベルとその言葉の二つの辞書の作成
#print(emoji_dict)
#print(df)
#print(df_none1)
#print(df_none2)
#print(kanzyo_emoji_dict1)
#print(kanzyo_emoji_dict2)
#print(kanzyo_emoji_dict3)
dict_emoji0=list(kanzyo_emoji_dict1.keys())
dict_emoji1=list(kanzyo_emoji_dict1.values())
dict_emoji2=list(kanzyo_emoji_dict2.values())
dict_emoji3=list(kanzyo_emoji_dict3.values())
for i in range(len(dict_emoji1)):
    dict_emoji1[i]=dict_emoji1[i]+dict_emoji2[i]+dict_emoji3[i]
#print(dict_emoji0)
#print(dict_emoji1)
dict_emoji0.pop()
dict_emoji1.pop()
kanzyo_emoji_dict=dict(zip(dict_emoji0,dict_emoji1))
#print(kanzyo_emoji_dict)
#感情語ラベルのカウントをし一番多いものにする。
for key,value in kanzyo_emoji_dict.items():
    mycounter=Counter(value)
    most1=mycounter.most_common(1)
    kanzyo_emoji_dict[key]=most1[0][0]

#print(kanzyo_emoji_dict)
#for i,w1 in enumerate(list(pn_answer_end.keys())):#辞書のキーで回す
        #print(w1)
        #if w1 in word_sent:#w1がword_sentのなかにあれば
            #word_sent=word_sent.replace(w1,dict1[i])#w1をdict[i]で置き換え

In [25]:
#Lex_Rankコード
mecab = MeCab.Tagger ("-Owakati -d /usr/local/mecab/lib/mecab/dic/mecab-ipadic-neologd/")
model_path = './word2vec/word2vec.gensim.model'
word2vec_model = gensim.models.Word2Vec.load(model_path)
def lex_rank(sentences, n, threshold,rate):
    """
    LexRankで文章を要約する．
    @param  sentences: list
        文章([[w1,w2,w3],[w1,w3,w4,w5],..]のような文リスト)
    @param  n: int
        文章に含まれる文の数
    @param  t: float
        コサイン類似度の閾値(default 0.1)
    @return : list
        LexRank
    """
    cosine_matrix = numpy.zeros((n, n))#n×nのゼロ行列二次元
    degrees = numpy.zeros((n,))#n×零行列
    l = []

     # 1. 隣接行列の作成
    for i in range(n):
        for j in range(n):
            cosine_matrix[i][j] = sentence_similarity(sentences[i], sentences[j])#隣接文章のコサイン類似度の行列
            if cosine_matrix[i][j] > threshold:#闘値より大きいならばコサイン類似度１degreeに＋１
                cosine_matrix[i][j] = 1
                degrees[i] += 1
            else:
                cosine_matrix[i][j] = 0#闘値以下なら０
    #print(cosine_matrix)
    #print(degrees)
    # 2.LexRank計算
    for i in range(n):
        for j in range(n):
            if degrees[i] != 0:
                cosine_matrix[i][j] = cosine_matrix[i][j] / degrees[i]#角度でコサイン類似度をわる（角度が大きいと類似度が小さい）
    #print(cosine_matrix)
    ratings=[]
    ratings=power_method(cosine_matrix,n,rate)
    #print(ratings)
    #print(sentences)
    #for i ,t in enumerate(ratings):
        #if(senti_score[i]>=0):
            #ratings[i]=math.sqrt(t*senti_score[i])#レイト付
        #elif(senti_score[i]<0):
            #ratings[i]=math.sqrt(t*senti_score[i]*-1)*-1
    
    #print(ratings)
    #print(ratings)

    return zip(sentences, ratings)

def avg_feature_vector(sentence, model, num_features):
    words = mecab.parse(sentence).replace(' \n', '').split() # mecabの分かち書きでは最後に改行(\n)が出力されてしまうため、除去
    feature_vec = numpy.zeros((num_features,), dtype="float32") # 特徴ベクトルの入れ物を初期化
    for word in words:
        try:
            feature_vec = numpy.add(feature_vec, model[word])
        except:
            continue
    #print(feature_vec)
    if len(words) > 0:
        feature_vec = numpy.divide(feature_vec, len(words))
    return feature_vec

sent_vec=[]
def sentence_similarity(sentence_1, sentence_2):
    # 今回使うWord2Vecのモデルは50次元の特徴ベクトルで生成されているので、num_featuresも50に指定
    num_features=50
    
    sentence_1_avg_vector = avg_feature_vector(sentence_1, word2vec_model, num_features)
    sentence_2_avg_vector = avg_feature_vector(sentence_2, word2vec_model, num_features)
    # １からベクトル間の距離を引いてあげることで、コサイン類似度を計算
    sent_vec.append(1 - spatial.distance.cosine(sentence_1_avg_vector, sentence_2_avg_vector))
    return 1 - spatial.distance.cosine(sentence_1_avg_vector, sentence_2_avg_vector)

def power_method(cosine_matrix, n, e):
    """
    べき乗法を行なう
    @param  scosine_matrix: list
        確率行列
    @param  n: int
        文章中の文の数
    @param  e: float
        許容誤差ε
    @return: list
        LexRank
    """
    transposed_matrix = cosine_matrix.T#転置
    #print(transposed_matrix)
    sentences_count = n#文章の数

    p_vector = numpy.array([1.0 / sentences_count] * sentences_count)#nの文章分の行列のアーレイ
    #print(p_vector)

    lambda_val = 1.0 #(ラムダの大きさ)

    while lambda_val > e:#ラムダの大きさが許容誤差より大きい場合
        next_p = numpy.dot(transposed_matrix, p_vector)#コサイン類似度の転置行列とPベクトルの内積
        #print(next_p)
        lambda_val = numpy.linalg.norm(numpy.subtract(next_p, p_vector))#next_p-p_vectorのノルム計算
        
        p_vector = next_p#次のPへ
    

    return p_vector

def mmr(lambda_, kotae,len_kotae):
    mmr_rating=[]
    Di=[]
    for i in range(len_kotae):#答えの長さで回す
        if Di != []:#選択された文章がある時
            use_mmr=[]
            for ds in range(len(DS)):#選択されていない文
                for di in range(len(Di)):#選択されている文
                    #print(DS[ds][0])
                    use_mmr.append(lambda_*DS[ds][1] - (1 - lambda_) * sentence_similarity(DS[ds][0],Di[di][0]))#mmr計算
                    #use_mmr=sorted(use_mmr)#mmrのソート
                    #mmr_rating.append(use_mmr[0])#mmr_ratingの更新
                    #use_mmr=[]
            use_kotae=sorted(list(zip(use_mmr,DS)))#mmrでソート(もっとも値が大きいものを選択するため)
            mmr_rating,use_kotae=list(zip(*use_kotae))#zipを解除
            use_kotae=list(use_kotae)#list化
            mmr_rating=list(mmr_rating)#list化
            DS=[]
            Di.append(use_kotae[0])#ソートの最大をDi(選択された文章に入れる)
            use_kotae.pop(0)#選択されたものをポップ
            for i in use_kotae:
                DS.append(i)#DS選択されていない文章の更新
        else:
            DS=[]
            Di.append(kotae[0])#選択される文にLexRankの結果を入れる
            for i in kotae:
                DS.append(i)#DSに入れる
            #print(DS)
            DS.pop(0)#DSのDiに加えたものをpopする。
            #print(DS)
        
    return Di

In [18]:
#semanticVここから

# MeCabで形態素に分割
def split_words(sentence):
    result = []
    available_pos = ["名詞", "動詞-自立", "形容詞"]
    not_available_pos = ["名詞-数"]
    tokenizer = MeCab.Tagger('-Ochasen -d /usr/local/mecab/lib/mecab/dic/mecab-ipadic-neologd/')
    chasen_result = tokenizer.parse(sentence)#解析結果を文字列で取得
    for line in chasen_result.split("\n"):#上のものを改行で分割
        elems = line.split("\t")#空白で区切る
        if len(elems) < 4:#長さが４以下ならば続行
            continue
        word = elems[0]#言葉
        pos = elems[3]#上記の分類
        if True in [pos.startswith(w) for w in not_available_pos]:#posがavaila---内の文字列で始まるかどうか
            continue
        if True in [pos.startswith(w) for w in available_pos]:#posがnot---内の文字列で始まるかどうか
            result.append(word)#結果に名詞、動詞-自立、形容詞を格納
    return result

# 文章をベクトルに変換（単語の平均ベクトルを用いる）
def wordvec2docmentvec(sentence,vec_model,num_features):
    doc_vec = np.zeros(num_features, dtype="float32")
    for word in sentence:
        try: #例外処理
            temp = vec_model[word]#modelのなかの単語のベクトルを抽出
        except:
            continue
        doc_vec += temp#文書のベクトルの合計を取る
    if len(sentence)>0:#センテンスの長さが０以上だったら
        doc_vec =  doc_vec / len(sentence)#文書の平均ベクトルを取る
        
    return doc_vec

# Proj を計算
def projection(u, b):
    return np.dot(u,b) * b #uとbの内積×b

# 基底ベクトル
def basis_vector(v):
    
    return v / np.linalg.norm(v)#vをVのノルムで割る(基底ベクトル)

#　Distance(u_i, B)
def span_distance(v, span_space):
    features=50
    proj = np.zeros(features, dtype="float32")#featureの数のゼロ行列
    for span_vec in span_space:
        proj += projection(v, span_vec)
    return np.linalg.norm(v - proj)#ベクトル間の距離

# index of sentence that is farthest from the subspace of Span(B).
def compute_farthest_spanspace(sentences_vector, span_subspace, skip_keys):
    distance = 0
    farthest_key = 0
    for i, vec in enumerate(sentences_vector): #インデックス付きのfor
        if i in skip_keys:#iにskip_keysが入っていたら続ける
            continue
        dist = span_distance(vec, span_subspace) #ベクトル間の長さ
        if dist >= distance:#もしも長さがdistanceより長かったら
            distance = dist
            farthest_key = i#もっとも大きいキー
    return farthest_key

def semanticVolume(sentence):
    original_data = []
    corpus = []
    corpus_vec = []
    model_path = './word2vec/word2vec.gensim.model'
    model = Word2Vec.load(model_path)#日本語の学習済みモデル
    features = 50

    for i in sentence:
        #print(i)
        h = split_words(i)
        if len(h) == 0:#hの長さが０ならばcontinue
            continue
        original_data.append(i)#original_dataにsentenceを入れる
        corpus.append(" ".join(h))#" h1 h2 h3"がかえる
        corpus_vec.append(wordvec2docmentvec(h, model, features))#corpusvecにベクトルを入れる
        #print(original_data1)
        
    L=3#要約文の数
    yoyaku_index = [] # 要約対象の文章ID
    # 重心を計算
    centroid = np.zeros(features, dtype="float32")
    for vec in corpus_vec:
        centroid += vec
    centroid /= len(corpus_vec)
    # index of sentence that is farthest from the cluster centroid
    distance = 0
    first_yoyaku_index = 0
    for i, vec in enumerate(corpus_vec): 
        dist = np.linalg.norm(centroid - vec)
        if dist >= distance:#距離が大きい場合
            distance = dist
            first_yoyaku_index = i#離れている文
    
    yoyaku_index.append(first_yoyaku_index)#文を作っていく
    # index of sentence that is farthest from s_p
    distance = 0
    second_yoyaku_index = 0
    
    for i, vec in enumerate(corpus_vec):
        if i in yoyaku_index:
            continue
        dist = np.linalg.norm(corpus_vec[first_yoyaku_index] - vec) #一番めのベクトルからベクトルを引く
        if dist >= distance:#距離が大きければ
            distance = dist
            second_yoyaku_index = i#離れている
    
    yoyaku_index.append(second_yoyaku_index)
    # total length
    total_length = len(original_data[first_yoyaku_index]) + len(original_data[second_yoyaku_index])#文の全体の長さ
    # first basis vector
    first_basis_vector = basis_vector(corpus_vec[second_yoyaku_index])#最初の基底ベクトル
    span_subspace = [first_basis_vector]#span_subspaceにbasisvectorを
    
    while(True):
        farthest_index = compute_farthest_spanspace(corpus_vec, span_subspace, yoyaku_index)
        if len(yoyaku_index) < L:#基底ベクトルの文とtotal_lengthの合計の文が200文字より小さければ
            span_subspace.append(corpus_vec[farthest_index])#span_subspaceにfarthest_indexのcorpus_vecをappendする
            total_length += len(original_data[farthest_index]) #全体の長さに最も遠いインデックスのオリジナルデータの長さをたす
            yoyaku_index.append(farthest_index)#要約の文にアテンド
        else:
            break
    
    kotae=[]
    indexs=[]
    for i in yoyaku_index:
        kotae.append(original_data[i])
        indexs.append(i)
    return zip(kotae,indexs)

#SemanticVここまで------------------------------------------------------------------------------------------------------------------------------------



In [19]:
#Topic_Rank

def js(topic1,topic2):
    m = (topic1+topic2)/2
    kl = entropy(topic1, topic2)
    kl_pm = entropy(topic1, m)
    kl_qm = entropy(topic2, m)
    js = (kl_pm + kl_qm)/2
    #print(js)
    return js

def power_method(topic_matrix, n, e):
    """
    べき乗法を行なう
    @param  scosine_matrix: list
        確率行列
    @param  n: int
        文章中の文の数
    @param  e: float
        許容誤差ε
    @return: list
        LexRank
    """
    transposed_matrix = topic_matrix.T#転置
    #print(transposed_matrix)
    sentences_count = n#文章の数

    p_vector = np.array([1.0 / sentences_count] * sentences_count)#nの文章分の行列のアーレイ
    #print(p_vector)

    lambda_val = 1.0 #(ラムダの大きさ)

    while lambda_val > e:#ラムダの大きさが許容誤差より大きい場合
        next_p = np.dot(transposed_matrix, p_vector)#コサイン類似度の転置行列とPベクトルの内積
        #print(next_p)
        lambda_val = np.linalg.norm(np.subtract(next_p, p_vector))#next_p-p_vectorのノルム計算
        #print(lambda_val)
        p_vector = next_p#次のPへ
        #print(p_vector)
    


    return p_vector

def topic_rank(num1, n, t,Rate,sentence):
    """
    LexRankで文章を要約する．
    @param  sentences: list
        文章([[w1,w2,w3],[w1,w3,w4,w5],..]のような文リスト)
    @param  n: int
        文章に含まれる文の数
    @param  t: float
        コサイン類似度の閾値(default 0.1)
    @return : list
        LexRank
    """
    test_topicRank=words[num1]
    topic=np.zeros((len(test_topicRank),8))
    #print(lda[corpus][1])
    for ei in range(len(test_topicRank)):
        for ii in range(len(lda[corpus][ei])):
            for si in range(len(lda[corpus][ei][ii])): 
                topic[ei][lda[corpus][ei][ii][0]]=lda[corpus][ei][ii][1]
    
    topic_matrix = np.zeros((n, n))#n×nのゼロ行列二次元
    degrees = np.zeros((n,))#n×零行列
    l = []

     # 1. 隣接行列の作成
    for i in range(n):
        for j in range(n):
            topic_matrix[i][j] = js(topic[i],topic[j])#隣接文章のtopic類似度の行列
            if topic_matrix[i][j] > t:#闘値より大きいならばコサイン類似度１degreeに＋１
                topic_matrix[i][j] = 1
                degrees[i] += 1
            else:
                topic_matrix[i][j] = 0#闘値以下なら０
    #print(topic_matrix)
    #print(degrees)
    # 2.LexRank計算
    for i in range(n):
        for j in range(n):
            if degrees[i] != 0:
                topic_matrix[i][j] = topic_matrix[i][j] / degrees[i]#角度でコサイン類似度をわる（角度が大きいと類似度が小さい）
    #print(topic_matrix)

    ratings = power_method(topic_matrix, n,Rate)#レイト付
   
    #print(ratings)
    #print(indexes)
    kotae=zip(sentence,ratings)
    #print(list(kotae))
    #indexes =mmr_sort(lambda_,kotae,limit)
    return kotae

def mmr(lambda_, kotae,len_kotae):
    mmr_rating=[]
    Di=[]
    for i in range(len_kotae):#答えの長さで回す
        if Di != []:#選択された文章がある時
            use_mmr=[]
            for ds in range(len(DS)):#選択されていない文
                for di in range(len(Di)):#選択されている文
                    #print(DS[ds][0])
                    use_mmr.append(lambda_*DS[ds][1] - (1 - lambda_) * sentence_similarity(DS[ds][0],Di[di][0]))#mmr計算
                    #use_mmr=sorted(use_mmr)#mmrのソート
                    #mmr_rating.append(use_mmr[0])#mmr_ratingの更新
                    #use_mmr=[]
            use_kotae=sorted(list(zip(use_mmr,DS)))#mmrでソート(もっとも値が大きいものを選択するため)
            mmr_rating,use_kotae=list(zip(*use_kotae))#zipを解除
            use_kotae=list(use_kotae)#list化
            mmr_rating=list(mmr_rating)#list化
            DS=[]
            Di.append(use_kotae[0])#ソートの最大をDi(選択された文章に入れる)
            use_kotae.pop(0)#選択されたものをポップ
            for i in use_kotae:
                DS.append(i)#DS選択されていない文章の更新
        else:
            DS=[]
            Di.append(kotae[0])#選択される文にLexRankの結果を入れる
            for i in kotae:
                DS.append(i)#DSに入れる
            #print(DS)
            DS.pop(0)#DSのDiに加えたものをpopする。
            #print(DS)
        
    return Di



In [20]:
#話し言葉、絵文字、敬語への変換

from pykakasi import kakasi
def conversion(text,switch):
    word_sent=""
    unuse_words=[]
    flag=0
    #text=normalize(text)
    #print(text)
    tagger = MeCab.Tagger ('-Ochasen /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
    result = tagger.parse("")
    node = tagger.parseToNode(text)
    tokens = list()
    idx = 0
    POS_DIC = {
            'BOS/EOS': 'EOS', # end of sentense
            '形容詞' : 'ADJ',
            '連体詞' : 'JADJ', # Japanese-specific POS like a adjective
            '副詞'   : 'ADV',
            '名詞'   : 'NOUN',
            '動詞'   : 'VERB',
            '助動詞' : 'AUX',
            '助詞'   : 'PART',
            '感動詞' : 'INTJ',
            '接続詞' : 'CONJ',
            '記号'   : 'SYM', # symbol
            '*'      : 'X',
            'その他' : 'X',
            'フィラー': 'X',
            '接頭詞' : 'X',
        }
    LOMA_TO_KANA={
        'ai':'い',
        'ii':'い',
        'ui':'い',
        'ei':'い',
        'oi':'い',
        'ki':'き',
        'si' : 'し',
        'ti' : 'ち',
        'ni' : 'に',
        'hi' : 'ひ',
        'mi' : 'み',
        'ri' : 'り',
        'gi' : 'ぎ',
        'zi' : 'じ',
        'ji' : 'じ',
        'di' : 'ぢ',
        'bi' : 'び',
        'pi' : 'ぴ',
        'ae':'え',
        'ie':'え',
        'ue':'え',
        'ee':'え',
        'oe':'え',
        'ke':'け',
        'se':'せ',
        'te':'て',
        'ne':'ね',
        'he':'へ',
        'me':'め',
        're':'れ',
        'ge' : 'げ',
        'ze' : 'ぜ',
        'de' : 'で',
        'be' : 'べ',
        'pe' : 'ぺ',
        }
    i=0
    answer_sent_1=[]
    while node :
        
        feature = node.feature.split(',')
        token = namedtuple('Token', 'idx, surface, pos, pos_detail1, pos_detail2, pos_detail3,\
        infl_type, infl_form, base_form, reading, phonetic')
        token.idx         = idx
        token.surface     = node.surface  # 表層形
        #print(node.surface)
        token.pos_jp      = feature[0]    # 品詞
        token.pos_detail1 = feature[1]    # 品詞細分類1
        token.pos_detail2 = feature[2]    # 品詞細分類2
        token.pos_detail3 = feature[3]    # 品詞細分類3
        token.infl_type   = feature[4]    # 活用型
        token.infl_form   = feature[5]    # 活用形
        token.base_form   = feature[6] if feature[6]!='*' else node.surface # 原型 ex)MacMini's base_form=='*'
        token.pos         = POS_DIC.get( feature[0], 'X' )     # 品詞
        token.reading     = feature[7] if len(feature) > 7 else ''  # 読み
        token.phonetic    = feature[8] if len(feature) > 8 else ''  # 発音
        #print(token.pos)
        #print(token.surface)
        #print(feature)
        # for BOS/EOS
        if token.pos != 'EOS':
            tokens.append(token)
            idx += 1
       
        #print("-----------------------------"+token.surface)
        
        
       
        #print(word_sent)
        if switch =="talk" or switch=="emoji":
            if (token.pos=="CONJ" or (token.pos=="PART" and token.pos_detail1=="接続助詞")) and token.surface in pn_answer_setu:
                token.surface=pn_answer_setu[token.surface]
                #print(token.surface)
            elif token.pos=="ADV" and token.surface in pn_answer_huku:
                token.surface=pn_answer_huku[token.surface]
                #print(token.surface)
            elif token.pos=="ADJ" and token.surface in pn_answer_keiyou:
                token.surface=pn_answer_keiyou[token.surface]
                #print(token.surface)
            elif token.pos=="PART" and token.surface in pn_answer_zyoshi:
                token.surface=pn_answer_zyoshi[token.surface]
            elif token.pos=="NOUN" and token.surface in pn_answer_meishi:
                token.surface=pn_answer_meishi[token.surface]
                #print("=====")
            dict1=list(pn_answer_end.values())
            if len(answer_sent_1)>2:
                if token.surface in ['。', '．', '！' ] and answer_sent_1[-1] not in ["か","なあ","や","ぞ","よ","さ","ね"] and tokens[-2].pos!="NOUN":
                    answer_sent_1.append("よ")
            answer_sent_1.append(token.surface)
            word_sent="".join(answer_sent_1)
            for i,w1 in enumerate(list(pn_answer_end.keys())):#辞書のキーで回す
                #print(w1)
                if w1 in word_sent:#w1がword_sentのなかにあれば
                    word_sent=word_sent.replace(w1,dict1[i])#w1をdict[i]で置き換え
                    
            #print(w1)
        if switch =="emoji":
            for i,w1 in enumerate(list(kanzyo_emoji_dict.keys())):#辞書のキーで回す
                if w1 in word_sent:#w1がword_sentのなかにあれば
                    for t in list(kanzyo_emoji_dict.values())[i]:
                        if t in emoji_dict and emoji_dict[t] not in unuse_words:
                            answer_sent_1.append(emoji_dict[t])
                            unuse_words.append(emoji_dict[t])
           
        if switch=="nomal":
            answer_sent_1.append(token.surface)
            word_sent="".join(answer_sent_1)
        node = node.next
        #print(tokens)
    #global kakasi
    if switch=="keigo" :
       

        if len(tokens)>=3:
            #print(tokens[-3].pos)
            for index in [-2,-3]:
                #print(tokens[-1].surface)
                if tokens[-1].surface in ['。', '．', '！' ,'！'] and tokens[index].pos=='VERB' :
                    global kakasi1
                    kakasi1 = kakasi()  # Generate kakasi instance
                    kakasi1.setMode("H", "a")  # Hiragana to ascii
                    kakasi1.setMode("K", "a")  # Katakana to ascii
                    kakasi1.setMode("J", "a")  # Japanese(kanji) to ascii
                    kakasi1.setMode("r", "Hepburn")  # Use Hepburn romanization
                    conv = kakasi1.getConverter()
                    kana=conv.do(tokens[index].base_form)
                    #print(index)
                    #print(tokens[index].infl_type[0:1])
                    if tokens[index].infl_type[0:2]=="五段" or tokens[index].infl_type[0:3]=="上一段":
                        kana_use=LOMA_TO_KANA[kana[-2]+"i"]
                        tokens[index].surface=tokens[index].surface[:-1]+kana_use
                    
                    elif tokens[index].infl_type[0:3]=="下一段":
                        kana_use=LOMA_TO_KANA[kana[-2]+"e"]
                        tokens[index].surface=tokens[index].surface[:-1]+kana_use
                    
                    elif tokens[index].infl_type[0:2]=="カ変" :
                        tokens[index].surface="来"
                    elif tokens[index].infl_type[0:2]=="サ変" :
                        tokens[index].surface="し"
                    elif tokens[index].infl_type[0:2]=="一段":
                    
                        if "る" in tokens[index].surface:
                            tokens[index].surface=tokens[index].surface.strip("る")
                
                    if index==-3 :
                        if tokens[-2].surface=='た' :
                            tokens[-2].surface='ました'
                       
                        elif tokens[-2].surface=='ない' :
                            tokens[-2].surface='ません'
                        else:
                            tokens[-2].surface='ます'
                        
                    elif index==-2 :
                        tokens[-1].surface='ます。'
                        break
                elif tokens[index].pos=="ADJ":
                    tokens[-1].surface='です。'
            #print(tokens[-1].surface)
            if tokens[-2].pos=="NOUN":
                    tokens[-1].surface="です。"
            
            
            if tokens[-2].surface=="だ":
                tokens[-2].surface="です"
        
        
            if len(tokens)>=4 and tokens[-4].surface+tokens[-3].surface=="だろう" and tokens[-2].surface=="か" and tokens[-1].surface in ["?","？"]:
                tokens[-4].surface="でしょう"
                tokens.pop(-3)
            elif tokens[-2].surface=="か" and tokens[-1].surface in ["?","？"]:
                tokens[-2].surface="でしょうか"
        
       
        
                #if(tokens[-2].surface=='た'):
                    #tokens[-1].surface='です。'
                #elif(tokens[-2].surface!='た'):
                    #tokens[index].surface=tokens[index].base_form
                    #if(index==-3):
                        #tokens[-2].surface='です'
                    #elif(index==-2):
                        #tokens[-1].surface='です。'
                    
                    
        for i in tokens:
            answer_sent_1.append(i.surface)
            word_sent="".join(answer_sent_1)
        #print(word_sent)
    return word_sent


In [21]:
shuffle=list(range(180))
for u in range(180):
    if(u in range(0,10)):
        shuffle[u]=37
    elif(u in range(10,20)):
        shuffle[u]=38
    elif(u in range(20,30)):
        shuffle[u]=39
    elif(u in range(30,40)):
        shuffle[u]=40
    elif(u in range(40,50)):
        shuffle[u]=41
    elif(u in range(50,60)):
        shuffle[u]=42
    elif(u in range(60,70)):
        shuffle[u]=43
    elif(u in range(70,80)):
        shuffle[u]=44
    elif(u in range(80,90)):
        shuffle[u]=45
    elif(u in range(90,100)):
        shuffle[u]=46
    elif(u in range(100,110)):
        shuffle[u]=47
    elif(u in range(110,120)):
        shuffle[u]=48
    elif(u in range(120,130)):
        shuffle[u]=49
    elif(u in range(130,140)):
        shuffle[u]=50
    elif(u in range(140,150)):
        shuffle[u]=51
    elif(u in range(150,160)):
        shuffle[u]=52
    elif(u in range(160,170)):
        shuffle[u]=53
    elif(u in range(170,180)):
        shuffle[u]=54

print(shuffle)
random.shuffle(shuffle)
print(shuffle)
print(len(shuffle))

[37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45, 45, 45, 45, 45, 45, 45, 45, 45, 45, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 47, 47, 47, 47, 47, 47, 47, 47, 47, 47, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 51, 51, 51, 51, 51, 51, 51, 51, 51, 51, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 54, 54, 54, 54, 54, 54, 54, 54, 54, 54]
[52, 40, 46, 49, 37, 42, 40, 48, 37, 52, 38, 53, 41, 51, 50, 39, 50, 37, 54, 40, 42, 41, 46, 43, 44, 43, 46, 49, 50, 42, 54, 51, 50, 38, 52, 51, 49, 54, 44, 51, 45, 47, 52, 44, 38, 42, 47, 53, 46, 45, 48, 49, 53, 37, 40, 37, 42, 45, 38, 38, 41, 43, 48, 53, 41, 54, 43, 50, 51, 37

In [22]:
#結果のファイル 化。
ritu=0.85
ikiti=0.7
rate=0.95
ikiti_topic=1.0
num_sent=360
count1=99
i=0
i_use=0
i2=0
www=0
abstway=0
for i2 in tqdm(range(180)):
    #print(i)
    i=i2+num_sent
    i_use=i2
    answer=[]
    #print("i2の結果:"+str(i2))
    #print("i_useの結果:"+str(i_use))
    #print(text_use[691])
    if(i_use in range(0,20)):
        answer=sorted(list(lex_rank(text_use[i],len(text_use[i]),ikiti,ritu)), key=lambda x:x[1], reverse=True)
        answer=mmr(0.5,answer,3)
        change="talk"
        way="Lex"
        abstway=0
        
        
    elif(i_use in range(20,40)):
        answer=sorted(list(lex_rank(text_use[i],len(text_use[i]),ikiti,ritu)), key=lambda x:x[1], reverse=True)
        answer=mmr(0.5,answer,3)
        change="emoji"
        way="Lex"
        abstway=1

    elif(i_use in range(40,60)):
        answer=sorted(list(lex_rank(text_use[i],len(text_use[i]),ikiti,ritu)), key=lambda x:x[1], reverse=True)
        answer=mmr(0.5,answer,3)
        change="keigo"
        way="Lex"
        abstway=2

    elif(i_use in range(60,80)):
        answer=sorted(list(topic_rank(i,len(text_use[i]),ikiti_topic,rate,text_use[i])), key=lambda x:x[1], reverse=True)
        answer=mmr(0.5,answer,3)
        change="talk"
        way="Topic"
        abstway=3

    elif(i_use in range(80,100)):
        answer=sorted(list(topic_rank(i,len(text_use[i]),ikiti_topic,rate,text_use[i])), key=lambda x:x[1], reverse=True)
        answer=mmr(0.5,answer,3)
        change="emoji"
        way="Topic"
        abstway=4

    elif(i_use in range(100,120)):
        answer=sorted(list(topic_rank(i,len(text_use[i]),ikiti_topic,rate,text_use[i])), key=lambda x:x[1], reverse=True)
        answer=mmr(0.5,answer,3)
        change="keigo"
        way="Topic"
        abstway=5

    elif(i_use in range(120,140)):
        answer=list(semanticVolume(text_use[i]))
        change="talk"
        way="SV"
        abstway=6

    elif(i_use in range(140,160)):
        answer=list(semanticVolume(text_use[i]))
        change="emoji"
        way="SV"
        abstway=7

    elif(i_use in range(160,180)):
        answer=list(semanticVolume(text_use[i]))
        change="keigo"
        way="SV"
        abstway=8
    
    if(answer!=[]):
        youyaku=[]
        text_answer=""
        for t in range(3):
            #print(answer[i][0])
            youyaku.append(conversion(answer[t][0],change))
            #print(youyaku)
            #print(texts_title[872])
        title=conversion(texts_title[i],change)
        text_answer = text_answer+str(abstway)+"\n"+title+"\n"
        text_answer=text_answer+"\n".join(youyaku)+"\n"
        text_answer=text_answer+"".join(texts_original[i])
        with open(str(shuffle[i2])+"/"+way+"_"+change+str(re.sub(r'[\\|/|:|?|.|"|<|>|\|]', '-', texts_title[i]))+".txt", "w") as f:  # w:上書きモード
            f.write(text_answer)

/Users/a.seki/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


FileNotFoundError: [Errno 2] No such file or directory: '52/Lex_talk日本3Dメガネドレッサー賞で仰天発言「3Dメガネで安めぐみの体を見たい」.txt'